In [1]:
import pip

package_names = ['lightgbm', 'awswrangler'] 
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


0

In [2]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
from lightgbm import LGBMClassifier

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)
#import scorecardpy as sc
#from utils import *
#from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

#from utils import *
#from modeler import *
#subfolder = "data"
# os.listdir(subfolder)

NumExpr defaulting to 4 threads.


In [3]:
# imports
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.amazon.amazon_estimator import get_image_uri
import awswrangler as wr
import pandas as pd
import sagemaker
import boto3

proyecto = 'propension'
normal = [1]
target = 'target_desembolso_f2m_mayor_30_menor_180'
clasif = 'normal' if len(normal) == 1 else 'all'

sess = sagemaker.session.Session()
bucket = sess.default_bucket() 
now = datetime.now()
role = sagemaker.get_execution_role()   ## levantamiento de roles
s3 = boto3.resource('s3')
region = boto3.Session().region_name
smclient = boto3.Session().client('sagemaker')

print("bucket: ", bucket)
print("region: ", region)
print("now: ", now)

bucket, region, target, clasif

bucket:  sagemaker-us-east-1-058528764918
region:  us-east-1
now:  2021-07-31 03:52:02.151049


('sagemaker-us-east-1-058528764918',
 'us-east-1',
 'target_desembolso_f2m_mayor_30_menor_180',
 'normal')

In [4]:
sufijo = '{}_clasif_{}'.format(target.lower(), clasif)
sufijo

'target_desembolso_f2m_mayor_30_menor_180_clasif_normal'

In [5]:
uri_raw = 's3://{}/vpc/{}/split'.format(bucket, proyecto)
uri_models = 's3://{}/vpc/{}/models'.format(bucket, proyecto)
uri_train = '{}/{}/train.csv'.format(uri_raw, sufijo)
uri_validation = '{}/{}/validation.csv'.format(uri_raw, sufijo)

print("uri_raw: ", uri_raw)
print("uri_train: ", uri_train)
print("uri_validation: ", uri_validation)
print("uri_models: ", uri_models)

uri_raw:  s3://sagemaker-us-east-1-058528764918/vpc/propension/split
uri_train:  s3://sagemaker-us-east-1-058528764918/vpc/propension/split/target_desembolso_f2m_mayor_30_menor_180_clasif_normal/train.csv
uri_validation:  s3://sagemaker-us-east-1-058528764918/vpc/propension/split/target_desembolso_f2m_mayor_30_menor_180_clasif_normal/validation.csv
uri_models:  s3://sagemaker-us-east-1-058528764918/vpc/propension/models


In [6]:
valid = pd.read_csv(uri_validation)
print(valid.shape)

del valid

(1750824, 16)


### MODELADO

In [7]:
container = get_image_uri(region, 'xgboost', '0.90-1')

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m5.4xlarge',
                                    output_path=uri_models,
                                    sagemaker_session=sess)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Same images used for training and inference. Defaulting to image scope: inference.
Defaulting to only available Python version: py3
Defaulting to only supported image scope: cpu.


In [28]:
xgb.set_hyperparameters(eval_metric = 'auc',   #'rmse',
                        objective = 'binary:logistic', #'reg:squarederror',          ---ojo se cambian ya que ahora el objetivo es clasiificacion
                        seed=SEED,
                        early_stopping_rounds = 5,
                        num_round = 1000,
                        #booster='dart',
                        alpha=1
                       )

hyperparameter_ranges = {
    #'scale_pos_weight': ContinuousParameter(0.015, 0.035),
    'colsample_bytree': ContinuousParameter(0.1, 0.75),
    'colsample_bynode': ContinuousParameter(0.1, 0.75),
    'colsample_bylevel': ContinuousParameter(0.1, 0.75),
    'max_depth': IntegerParameter(2, 3),
    'eta': ContinuousParameter(0.01, 0.15),
    #'rate_drop': ContinuousParameter(0.1, 0.5),
    #'skip_drop': ContinuousParameter(0.1, 0.5),
    #'one_drop': IntegerParameter(0, 1),
    #'sample_type': CategoricalParameter(['uniform', 'weighted']),
    #'normalize_type': CategoricalParameter(['tree', 'forest']),
    #'tree_method': CategoricalParameter(['approx', 'hist']),
    'subsample': ContinuousParameter(0.1, 0.9),
    'gamma': ContinuousParameter(0, 10),
    'max_delta_step': IntegerParameter(0, 5)
}

In [29]:
objective_metric_name = 'validation:auc'

In [30]:
tuner = HyperparameterTuner(
    xgb,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=30,
    max_parallel_jobs=10,
    objective_type='Maximize',   #Minimize OJO!!! era mínimo ya que mientras menor RMSE mejor modelo de regresion, acá cambia /// mayor AUc mejor modelo 
    base_tuning_job_name='xgboost-vpc-7'
)

In [31]:
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data=uri_train, 
    content_type='csv'
)   # OJO!! revisar cambio por la version

s3_input_validation = sagemaker.inputs.TrainingInput(
    s3_data=uri_validation, 
    content_type='csv'
) # OJO!! revisar cambio por la version

In [32]:
tuner.fit(
    {
        'train': s3_input_train, 
        'validation': s3_input_validation
    }, 
    include_cls_metadata=False
)

Defaulting to the only supported framework/algorithm version: latest.
Ignoring unnecessary instance type: None.
Creating hyperparameter tuning job with name: xgboost-vpc-7-210731-0433
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


In [33]:
print(tuner.latest_tuning_job.name)

xgboost-vpc-7-210731-0433


In [34]:
tuner.best_training_job()

'xgboost-vpc-7-210731-0433-029-54f28ca2'

In [35]:
tuner.describe()

{'HyperParameterTuningJobName': 'xgboost-vpc-7-210731-0433',
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:058528764918:hyper-parameter-tuning-job/xgboost-vpc-7-210731-0433',
 'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian',
  'HyperParameterTuningJobObjective': {'Type': 'Maximize',
   'MetricName': 'validation:auc'},
  'ResourceLimits': {'MaxNumberOfTrainingJobs': 30,
   'MaxParallelTrainingJobs': 10},
  'ParameterRanges': {'IntegerParameterRanges': [{'Name': 'max_depth',
     'MinValue': '2',
     'MaxValue': '3',
     'ScalingType': 'Auto'},
    {'Name': 'max_delta_step',
     'MinValue': '0',
     'MaxValue': '5',
     'ScalingType': 'Auto'}],
   'ContinuousParameterRanges': [{'Name': 'colsample_bytree',
     'MinValue': '0.1',
     'MaxValue': '0.75',
     'ScalingType': 'Auto'},
    {'Name': 'colsample_bynode',
     'MinValue': '0.1',
     'MaxValue': '0.75',
     'ScalingType': 'Auto'},
    {'Name': 'colsample_bylevel',
     'MinValue': '0.1',
     'MaxVal

In [36]:
nombre_exportado = HyperparameterTuner.attach(tuner.latest_tuning_job.name).best_training_job()
nombre_exportado

'xgboost-vpc-7-210731-0433-029-54f28ca2'

In [37]:
from sagemaker.estimator import Estimator, Transformer
model = Estimator.attach(nombre_exportado)


2021-07-31 05:19:10 Starting - Preparing the instances for training
2021-07-31 05:19:10 Downloading - Downloading input data
2021-07-31 05:19:10 Training - Training image download completed. Training in progress.
2021-07-31 05:19:10 Uploading - Uploading generated training model
2021-07-31 05:19:10 Completed - Training job completed


In [38]:
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.name)
tuner_metrics

<sagemaker.HyperparameterTuningJobAnalytics for xgboost-vpc-7-210731-0433>

In [39]:
results = tuner_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=False)
results.head(10)

,colsample_bylevel,colsample_bynode,colsample_bytree,eta,gamma,max_delta_step,max_depth,subsample,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
1,0.572882,0.705468,0.750000,0.109698,2.407630,5.0,3.0,0.851909,xgboost-vpc-7-210731-0433-029-54f28ca2,Completed,0.805077,2021-07-31 04:44:25+00:00,2021-07-31 05:19:10+00:00,2085.0
6,0.716114,0.695385,0.668653,0.011219,6.391199,5.0,3.0,0.203585,xgboost-vpc-7-210731-0433-024-8ceec75b,Completed,0.781476,2021-07-31 04:43:20+00:00,2021-07-31 04:44:58+00:00,98.0
4,0.716114,0.695385,0.668653,0.011219,6.391199,5.0,3.0,0.219585,xgboost-vpc-7-210731-0433-026-560ecd89,Completed,0.781188,2021-07-31 04:43:47+00:00,2021-07-31 04:44:57+00:00,70.0
5,0.716114,0.695385,0.668653,0.011219,6.191199,5.0,3.0,0.219585,xgboost-vpc-7-210731-0433-025-ce8da7ea,Completed,0.781188,2021-07-31 04:43:57+00:00,2021-07-31 04:45:06+00:00,69.0
25,0.589979,0.491422,0.476479,0.073590,5.481395,5.0,3.0,0.825761,xgboost-vpc-7-210731-0433-005-c56021bb,Completed,0.771370,2021-07-31 04:36:18+00:00,2021-07-31 04:37:54+00:00,96.0
9,0.328788,0.520943,0.677500,0.062446,6.570032,3.0,3.0,0.864946,xgboost-vpc-7-210731-0433-021-8c8cdc08,Completed,0.769523,2021-07-31 04:43:32+00:00,2021-07-31 04:45:22+00:00,110.0
11,0.328788,0.520943,0.677500,0.062446,6.370032,3.0,3.0,0.864946,xgboost-vpc-7-210731-0433-019-d296a7b0,Completed,0.769520,2021-07-31 04:40:12+00:00,2021-07-31 04:41:36+00:00,84.0
29,0.243763,0.412206,0.353736,0.120712,3.606873,3.0,3.0,0.621839,xgboost-vpc-7-210731-0433-001-468affa0,Completed,0.760779,2021-07-31 04:35:53+00:00,2021-07-31 04:37:50+00:00,117.0
8,0.483493,0.662654,0.425987,0.040127,5.605676,5.0,3.0,0.710860,xgboost-vpc-7-210731-0433-022-5e973b46,Completed,0.760436,2021-07-31 04:43:25+00:00,2021-07-31 04:44:48+00:00,83.0
24,0.100260,0.577896,0.615097,0.085771,4.261412,4.0,3.0,0.830519,xgboost-vpc-7-210731-0433-006-52001c5b,Completed,0.760384,2021-07-31 04:36:36+00:00,2021-07-31 04:37:37+00:00,61.0
